https://stackoverflow.com/questions/12761510/python-how-can-i-calculate-the-average-word-length-in-a-sentence-using-the-spl

https://pythonprogramming.net/combine-classifier-algorithms-nltk-tutorial/?completed=/sklearn-scikit-learn-nltk-tutorial/

http://streamhacker.com/2012/11/22/text-classification-sentiment-analysis-nltk-scikitlearn/

http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html#sample-pipeline-for-text-feature-extraction-and-evaluation

http://www.aicbt.com/authorship-attribution/

### Import

In [2]:
from IPython.core.display import HTML
HTML("<style>.container { width:75% !important; }</style>")
# HTML("<style>div.cell.selected {border-left-width: 1px !important;}</style>")

In [3]:
from __future__ import print_function

import pandas as pd
import numpy as np
from numpy import *
import re, reprlib, sys, csv, logging, subprocess
import matplotlib.pyplot as plt
from itertools import *
import random as ran


import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

ran.seed(5125)

from pprint import *
from time import *
from rpy2 import *

%matplotlib inline
%matplotlib notebook

from scipy.cluster.vq import *

import nltk as n
import nltk, nltk.classify.util, nltk.metrics, nltk.tokenize, nltk.stem
from nltk.corpus import *
from nltk.stem import *
from nltk.classify import *
from nltk.collocations import *
from nltk.metrics import *
from nltk.probability import *
from nltk.classify.scikitlearn import *
from nltk.tag.sequential import *
from nltk.tag.util import *
from nltk.tag import *

from sklearn_pandas import *
import sklearn as sk
from sklearn import *
from sklearn.feature_extraction.text import *
from sklearn.svm import *
from sklearn.cross_validation import *
from sklearn.pipeline import *
from sklearn.multiclass import *
from sklearn.datasets import *
from sklearn.naive_bayes import *
from sklearn.neighbors import *
from sklearn.cluster import *
from sklearn.feature_selection import *
from sklearn.ensemble import *
from sklearn.linear_model import *
from sklearn.tree import *
from sklearn.grid_search import *
from sklearn.base import *
from sklearn.datasets.twenty_newsgroups import *
from sklearn.decomposition import *
from sklearn.feature_extraction import *
from sklearn.metrics import *

/usr/local/lib/python3.4/dist-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


### Execute R script

In [3]:
# http://www.mango-solutions.com/wp/2015/10/integrating-python-and-r-part-ii-executing-r-from-python-and-vice-versa/
command = 'Rscript'
path2script = 'lexical_features.R'

# Build subprocess command
cmd = [command, path2script]

subprocess.check_output(cmd, universal_newlines=True)

CalledProcessError: Command '['Rscript', 'lexical_features.R']' returned non-zero exit status 2

In [4]:
data = pd.read_csv("../raw_data/data.csv", parse_dates=True, infer_datetime_format=True, 
            sep = None, encoding = "latin-1", engine = "python")
data.head(n=1)

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180,14861.6,93.29,0.03,15661,0.49,0.1


In [5]:
data_utf8 = pd.read_csv("../raw_data/data_utf8.csv", parse_dates=True, infer_datetime_format=True)
data_utf8.head(n=1)
data_utf8.shape

(9917, 20)

In [6]:
word = "likes the sound of thunder."
print(sum(c != ' ' for c in word))
# data["Lenght_of_Rows_withSpaces"] = np.nan
# data["Lenght_of_Rows_withoutSpaces"] = np.nan
# data["Word_Lenght"] = np.nan

23


In [7]:
d = {'n': "no", 'y': "yes"}

# http://stackoverflow.com/a/17702781
data = data.replace(d)

neu = data[["#AUTHID","STATUS","cNEU"]]
ext = data[["#AUTHID","STATUS","cEXT"]]
agr = data[["#AUTHID","STATUS","cAGR"]]
con = data[["#AUTHID","STATUS","cCON"]]
opn = data[["#AUTHID","STATUS","cOPN"]]

In [8]:
train_neu1, test_neu1 = sk.cross_validation.train_test_split(neu, train_size = 0.66, stratify= neu["cNEU"])
train_neu1.stack().value_counts().head()

no                                  4093
yes                                 2453
6f2bebc01062eb8334dccba3e048fdb5     161
e6cdef6f475cce3023c5b715f8c9f110     150
527ed53d2ba3a3bc417b8402d5b2f556     130
dtype: int64

In [9]:
train_neu, test_neu = sk.cross_validation.train_test_split(neu, train_size = 0.66)
train_neu.stack().value_counts().head()

no                                  4108
yes                                 2437
e6cdef6f475cce3023c5b715f8c9f110     150
6f2bebc01062eb8334dccba3e048fdb5     139
527ed53d2ba3a3bc417b8402d5b2f556     136
dtype: int64

In [10]:
train_neu, test_neu = sk.cross_validation.train_test_split(neu, train_size = 0.66, stratify= neu["cNEU"])
train_ext, test_ext = sk.cross_validation.train_test_split(ext, train_size = 0.66, stratify= ext["cEXT"])
train_agr, test_agr = sk.cross_validation.train_test_split(agr, train_size = 0.66, stratify= agr["cAGR"])
train_con, test_con = sk.cross_validation.train_test_split(con, train_size = 0.66, stratify= con["cCON"])
train_opn, test_opn = sk.cross_validation.train_test_split(opn, train_size = 0.66, stratify= opn["cOPN"])

### Grid Search for best parameters

In [11]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

classifierNB = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,2))),
    ('nb', LinearSVC())])

In [12]:
# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}

In [13]:
# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
grid_search.fit(neu["STATUS"], neu["cNEU"])
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

[Parallel(n_jobs=-1)]: Done  49 tasks       | elapsed:   29.7s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:   45.8s finished


Performing grid search...
pipeline: ['vect', 'tfidf', 'clf']
parameters:
{'clf__alpha': (1e-05, 1e-06),
 'clf__penalty': ('l2', 'elasticnet'),
 'vect__max_df': (0.5, 0.75, 1.0),
 'vect__ngram_range': ((1, 1), (1, 2))}
Fitting 3 folds for each of 24 candidates, totalling 72 fits
done in 47.210s

Best score: 0.560
Best parameters set:
	clf__alpha: 1e-06
	clf__penalty: 'elasticnet'
	vect__max_df: 0.75
	vect__ngram_range: (1, 2)


### Lexical and other features

In [35]:
def SyntacticFeatures(chapters):
    """
    Extract feature vector for part of speech frequencies
    """
    list_of_rows_CD = [l.split("\n")[0] for l in chapters]
    def token_to_pos(ch):
        tokens = n.word_tokenize(ch)
        #print(tokens)
        return [p[1] for p in n.pos_tag(tokens)]

    chapters_pos = [token_to_pos(kCD) for kCD in list_of_rows_CD]
    print(chapters_pos)
    pos_list = ['NN', 'NNP', 'DT', 'IN', 'JJ', 'NNS']
    fvs_syntax = np.array([[ch.count(pos) for pos in pos_list]
                           for ch in chapters_pos]).astype(np.float64)
    # normalise by dividing each row by number of tokens in the chapter
    fvs_syntax /= np.c_[np.array([len(ch) for ch in chapters_pos])]
    return fvs_syntax

feature_sets = None
feature_sets = list(SyntacticFeatures(neu["STATUS"]))

KeyboardInterrupt: 

In [36]:
def LexicalFeatures(chapters):
    sentence_tokenizer = n.data.load('tokenizers/punkt/english.pickle')
    word_tokenizer = n.tokenize.RegexpTokenizer(r'\w+')
    # create feature vectors
    num_chapters = len(chapters)
    fvs_lexical = np.zeros((len(chapters), 3), np.float64)
    fvs_punct = np.zeros((len(chapters), 3), np.float64)
    for e, ch_text in enumerate(chapters):
        # note: the nltk.word_tokenize includes punctuation
        tokens = nltk.word_tokenize(ch_text.lower())
        words = word_tokenizer.tokenize(ch_text.lower())
        sentences = sentence_tokenizer.tokenize(ch_text)
        vocab = set(words)
        words_per_sentence = np.array([len(word_tokenizer.tokenize(s))
                                       for s in sentences])

        # average number of words per sentence
        fvs_lexical[e, 0] = words_per_sentence.mean()
        # sentence length variation
        fvs_lexical[e, 1] = words_per_sentence.std()
        # Lexical diversity
        fvs_lexical[e, 2] = len(vocab) / float(len(words))

        # Commas per sentence
        fvs_punct[e, 0] = tokens.count(',') / float(len(sentences))
        # Semicolons per sentence
        fvs_punct[e, 1] = tokens.count(';') / float(len(sentences))
        # Colons per sentence
        fvs_punct[e, 2] = tokens.count(':') / float(len(sentences))

    # apply whitening to decorrelate the features
    fvs_lexical = whiten(fvs_lexical)
    fvs_punct = whiten(fvs_punct)

### Train all new features using Multinomial BN

In [37]:
data_n = pd.read_csv("../raw_data/data_n.csv", parse_dates=True, infer_datetime_format=True)
data_n.head(n=1)

,Unnamed: 0,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,...,Number_of_Dots,Number_of_Commas,Number_of_Semicolons,Number_of_Colons,Average_Word_Length,Lexical_Diversity,Number_of_FunctionalWords,Number_of_Pronouns,Number_of_PROPNAMEs,SentimentNumeric
0,1,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,...,1,0,0,0,4.4,0.7,2,0,0,2


In [38]:
d = {'n': 0, 'y': 1}
# http://stackoverflow.com/a/17702781
data_n = data_n.replace(d)
new_fea = data_n[["STATUS","cCON","cOPN","StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Avarage_Word_Lenght", "Lexical_Diversity"]]
#a,b, = new_fea.data.shape
new_fea.head(n=1)

KeyError: "['Avarage_Word_Lenght'] not in index"

In [39]:
train_new_fea, test_new_fea = sk.cross_validation.train_test_split(new_fea, train_size = 0.66)

NameError: name 'new_fea' is not defined

In [40]:
#clas_fit_NB = classifierNB.fit(train_new_fea[["Number_of_Commas", "StringLength"]], train_new_fea["cOPN"])

model = LinearSVC()
model.fit(train_new_fea[["Number_of_Commas", "Number_of_Semicolons","Number_of_Colons",  "StringLength",
                         "Avarage_Word_Lenght", 
                         "Number_of_Words","Number_of_Dots", 
                         "Lexical_Diversity"]], train_new_fea["cOPN"])
predicted = model.predict(test_new_fea[["Number_of_Commas", "Number_of_Semicolons","Number_of_Colons", 
                                        "Avarage_Word_Lenght","StringLength",
                                        "Number_of_Words","Number_of_Dots", "Lexical_Diversity"]])
x = test_new_fea[["cOPN","cCON"]]

def merge(a, b):
    val = 0
    if a == 0 and b == 0:
        val = 0
    if a == 0 and b == 1:
        val = 1
    if a == 1 and b == 0:
        val = 2
    if a == 1 and b == 1:
        val = 3
    return val
    pass

personality_trait_comb = [ merge(l[0], l[1]) for l in test_new_fea[["cOPN", "cCON"]].values]

print(metrics.classification_report(personality_trait_comb, predicted))
# print(metrics.confusion_matrix(personality_trait_comb, predicted))

#predicted_NB = classifierNB.predict(test_new_fea[["cCON", "cOPN"]])

NameError: name 'train_new_fea' is not defined

In [41]:
scores_NB = cross_validation.cross_val_score(model, data_n[["Number_of_Commas", "Number_of_Semicolons","Number_of_Colons",  "StringLength","Avarage_Word_Lenght", "Number_of_Words","Number_of_Dots", "Lexical_Diversity"]], data_n["cOPN"], cv = 10)
print("Accuracy of Linear SVC: %0.3f (+/- %0.2f)" % (scores_NB.mean(), scores_NB.std() * 2))

KeyError: "['Avarage_Word_Lenght'] not in index"

### Feature Union (Pipeline)

In [42]:
class ColumnSelector(TransformerMixin):
    """
    Class for building sklearn Pipeline step. 
    This class should be used to select a column from a pandas data frame.
    """

    def __init__(self, column):
        self.column = column

    def fit(self, x, y=None):
        return self

    def transform(self, data_frame):
        return data_frame[[self.column]]
    
pipelineUnion = Pipeline([
    # Use FeatureUnion to combine the features from subject and body
    ('union', FeatureUnion(
        transformer_list=[

            ('STATUS', Pipeline([
                ('extract', ColumnSelector(column="STATUS")),
                ('tfidf', TfidfVectorizer(ngram_range = (1,2)))
            ])),

            ('Number_of_Commas', Pipeline([
                ('extract', ColumnSelector(column="Number_of_Commas"))
            ])),

        ],

        # weight components in FeatureUnion
        transformer_weights={
            'STATUS': 0.6,
            'Number_of_Commas': 0.5,
        },
    )),

    # Use a SVC classifier on the combined features
    ('lsvc', LinearSVC()),
])

### Execute

In [43]:
# "Number_of_Semicolons","Number_of_Colons", "StringLength", "Avarage_Word_Lenght", "Number_of_Words","Number_of_Dots", "Lexical_Diversity"
# pipelineUnion.fit(train_new_fea, train_new_fea["cOPN"])

#train_new_fea[["STATUS", "Number_of_Commas"]], train_new_fea["cOPN"].values
#y = pipelineUnion.predict(test_new_fea[["Number_of_Commas", "Number_of_Semicolons","Number_of_Colons",  "StringLength",
#                          "Avarage_Word_Lenght", 
#                          "Number_of_Words","Number_of_Dots", 
#                          "Lexical_Diversity"]])
#print(classification_report(y, test.target))

In [44]:
y_actu = [ 1,  1, 0, 0, 0,  1, 0,  1,  1,
           0,  1, 0, 0, 0, 0, 0,  1, 0,
            1,  1,  1,  1, 0, 0, 0,  1, 0,
            1, 0, 0, 0, 0,  1,  1, 0, 0,
           0,  1,  1,  1,  1, 0, 0, 0, 0,
            1, 0, 0, 0, 0, 0, 0, 0, 0,
           0,  1,  1, 0,  1, 0,  1,  1,  1,
           0, 0,  1, 0,  1, 0, 0,  1, 0,
           0, 0, 0, 0, 0, 0, 0,  1, 0,
            1,  1,  1,  1, 0, 0,  1, 0,  1,
            1, 0,  1, 0,  1, 0, 0,  1,  1,
           0, 0,  1,  1, 0, 0, 0, 0, 0,
           0,  1,  1, 0]

y_pred = [0, 0, 0, 0, 0,  1, 0, 0,  1,
       0,  1, 0, 0, 0, 0, 0, 0, 0,
        1,  1,  1,  1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0,  1, 0, 0, 0,
       0,  1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0,
       0,  1, 0, 0, 0, 0, 0, 0, 0,
       0, 0,  1, 0, 0, 0, 0,  1, 0,
       0, 0, 0, 0, 0, 0, 0,  1, 0,
       0,  1, 0, 0, 0, 0,  1, 0,  1,
        1, 0, 0, 0,  1, 0, 0,  1,  1,
       0, 0,  1,  1, 0, 0, 0, 0, 0,
       0,  1, 0, 0]

# binary_confusion_matrix = BinaryConfusionMatrix(y_actu, y_pred)
# # print("Binary confusion matrix:\n%s" % binary_confusion_matrix)
# binary_confusion_matrix.print_stats()
# print(sk.metrics.average_precision_score(y_actu, y_pred))
# print(sk.metrics.accuracy_score(y_actu, y_pred, normalize=True))
# print(sk.metrics.mean_absolute_error(y_actu, y_pred))
# # print(y_actu.shape, y_pred.shape)
# print(sk.metrics.classification_report(y_actu, y_pred))

#### Testing

Source: https://stackoverflow.com/questions/25250654/how-can-i-use-a-custom-feature-selection-function-in-scikit-learns-pipeline

http://scikit-learn.org/stable/auto_examples/feature_stacker.html

http://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection

http://zacstewart.com/2014/08/05/pipelines-of-featureunions-of-pipelines.html

In [20]:
data_n = pd.read_csv("../raw_data/data_n.csv", parse_dates=True, infer_datetime_format=True)
d = {'n': 0, 'y': 1} # 1, y, = TRUE             0, n = FALSE
data_n = data_n.replace(d)
data_n.head(n=1)

,Unnamed: 0,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,...,Number_of_Dots,Number_of_Commas,Number_of_Semicolons,Number_of_Colons,Average_Word_Length,Lexical_Diversity,Number_of_FunctionalWords,Number_of_Pronouns,Number_of_PROPNAMEs,SentimentNumeric
0,1,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,0,1,...,1,0,0,0,4.4,0.7,2,0,0,2


In [21]:
columns_data = ["STATUS", "StringLength", 
          "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
          "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", 
          "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", 
          "SentimentNumeric"]

In [22]:
neu = data_n[["#AUTHID","STATUS","cNEU", "cAGR", "cOPN","StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]]

In [34]:
train_feat_neu, test_feat_neu, train_class_neu, test_class_neu = sk.cross_validation.train_test_split(neu, 
                                    neu["cNEU"], train_size = 0.66, stratify = neu["cNEU"], random_state= 5152)

print(train_feat_neu.shape)
print(test_feat_neu.shape)

(6545, 17)
(3372, 17)


In [35]:
# print(train_feat_neu.iloc[:, 1])

In [36]:
%run ETL.py

In [37]:
pipeline = Pipeline([     
    ('features', FeatureUnion(
          transformer_list=[
            ('ngram_tf_idf', Pipeline([
              ('selector', ColumnSelector()),
              ('counts', TfidfVectorizer(ngram_range = (1,2))),
            ])),

            ('rest', Pipeline([
              ('extract', ColumnExtractor()), 
              ('best', TruncatedSVD(n_components=9, random_state = 5152)),
            ])),   
        ],

          transformer_weights={
            'ngram_tf_idf': 1.0,
            'rest': 0.5,
          },
    )),
    ('classifier', LinearSVC()),
])

In [38]:
%%capture capt
pipeline.fit(train_feat_neu[columns_data], train_class_neu)

Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('ngram_tf_idf', Pipeline(steps=[('selector', <__main__.ColumnSelector object at 0x7f2398aceba8>), ('counts', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [39]:
y = pipeline.predict(test_feat_neu[columns_data])
# print(classification_report(y, test_class_neu, labels=[0, 1], target_names=["0", "1"]))

In [40]:
print("F1: ", sk.metrics.f1_score(test_class_neu, y, labels=[0, 1], average='binary'))
print("Precision: ", sk.metrics.average_precision_score(test_class_neu, y, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_neu, y, labels=[0, 1], average='binary'))

F1:  0.357924797715
Precision:  0.505018483563
Recall:  0.29746835443


In [41]:
# print(train_feat_neu[columns_data])